In [2]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun
import OLS_Estimation as OLS


In [3]:
df = pd.read_csv('Dataset.csv')
df_straf = df.copy()
df_straf = fun.straf_0ms(df_straf)
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
print(f'shape: {df.shape}')
df.head(11)


c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = -df['Price'][i]*1000
c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HP'][i] = -df['HP'][i]
c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

shape: (1232, 14)


,ID,Year,Sales,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,const
0,1,2013,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
1,1,2014,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
2,1,2015,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
3,1,2016,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
4,1,2017,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
5,1,2018,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
6,1,2019,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
7,1,2020,0,0.000000,Aiways,U5,400,-3.308258e+08,-201,-34,SUV,C,CH,1.0
8,1,2021,257,0.010319,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
9,1,2022,183,0.005938,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0


# Creating dummies

In [4]:
df2 = df.copy()
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)
df2['China'] = (df2['Country'] == 'CH').astype(int)

# Creating log market share

In [5]:
df2['Market share'] = df2['Market share'].replace(0, 1)
df2['log_market_share'] = np.log(df2['Market share']) 
df2['log_market_share'].head(5)

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: log_market_share, dtype: float64

# Independent OLS

In [6]:
data = df2

print(f'shape: {data.shape}')
#data format
N = data['ID'].unique().size
T = df['Year'].unique().size
print(f'Data has N={N} and T={T} data is = {data.shape[0]}, {N*T}')
assert data.shape[0] == N*T, f'Error: data is not a balanced panel'

variables = ['Price', 'HP', 'Chargetime', 'Range','Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J', 'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023', 'China']

#Declare variables
y = data["log_market_share"].values.reshape((N*T,1))
X = np.hstack([np.ones((N*T, 1))] + [data[var].values.reshape((N*T, 1)) for var in variables])
xnames = ['Cons'] + variables
print(f'Variables: {len(variables)}')


# Estimate with OLS
results = fun.est_OLS(y, X, xnames)
# fun.logit(y, X, xnames)


shape: (1232, 31)
Data has N=112 and T=11 data is = 1232, 1232
Variables: 22


In [17]:
#X skal være [i:i+1,2:] 2: for at fjerne consten og pris
#beta skal være [2:] for at fjerne konstant og pris-koefficienten og reshapes for at kunne lave matrixmultiplikaiton. 

#coefficients = OLS.get_coefficients_OLS(pd.read_csv('Dataset.csv'))
coefficients = results.params
print(f'coefficients: {coefficients[:10]}, shape: {coefficients.shape}')

coefficients_labels = xnames[1:] #Fjerne konstanten

model_labels = data['Model'].unique()
j = 0
for i in model_labels:
    j = j
    #print(f'Car {j}: {i}')
    j = j+1  
print(f'models: {model_labels[:3]}')

repeated_models = np.repeat(model_labels, T)

alpha = coefficients[1]
beta = coefficients
p_j = df_straf['Price'].values.reshape((N*T,1)) 

coefficients: [-1.56444827e+00 -4.64080954e-09  3.28080245e-04 -1.45454897e-02
  2.48323848e-04 -2.13594306e-01  3.99112793e-02 -3.50650084e-01
 -1.33903078e+00 -2.75422093e+00], shape: (23,)
models: ['U5' 'U6' 'e-tron']


In [19]:
ccp_list = fun.ccp(alpha, beta, X, p_j)
print(f'ccp_list: {len(ccp_list)}')
#print(f'ccp_list: {ccp_list.shape}')

Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Price:[[-3.30825789e+08]
 [ 3.30825789e+05]
 [ 3.30825789e+05]
 [ 3.30825789e+05]]
Pric

In [8]:
m = 3

In [9]:
fun.probability_ratio(ccp_list[:m], repeated_models[:m], repeated_models[:m])

,U5,U5,U5
U5,[[1.0]],[[0.9589070565392487]],[[1.0031508878463373]]
U5,[[1.042853937908287]],[[1.0]],[[1.0461398537067472]]
U5,[[0.996859009063829]],[[0.9558951381660284]],[[1.0]]


In [10]:
fun.marginal_effects(ccp_list[:m], repeated_models[:m], coefficients_labels[:m], coefficients[:m])

,Price,HP,Chargetime
U5,[[-5.279617962689559e-12]],[[3.732223223479503e-07]],NaN
U5,[[-5.505870383042135e-12]],[[3.892163685758361e-07]],NaN
U5,[[-5.263034730522306e-12]],[[3.720500344162787e-07]],NaN


In [11]:
fun.cross_marginal_effects(ccp_list[:m], coefficients[:m])

(3, 3, 2)


array([[[ 6.02006377e-15, -4.25586092e-10],
        [ 6.27804720e-15, -4.43824131e-10],
        [ 6.00115480e-15, -4.24249329e-10]],

       [[ 6.27804720e-15, -4.43824131e-10],
        [ 6.54708625e-15, -4.62843743e-10],
        [ 6.25832791e-15, -4.42430084e-10]],

       [[ 6.00115480e-15, -4.24249329e-10],
        [ 6.25832791e-15, -4.42430084e-10],
        [ 5.98230523e-15, -4.22916766e-10]]])

In [12]:
fun.elasticity(ccp_list[:m], repeated_models[:m], coefficients_labels[:m], coefficients[:m], X[:m])

elasticity shape: 
(3, 3)


,Price,HP,Chargetime
U5,[[1.533550854063779]],[[0.0658690222799449]],NaN
U5,[[1.5334759185806208]],[[0.06586580365371113]],NaN
U5,[[1.5335563464802262]],[[0.06586925819002273]],NaN


In [13]:
fun.cross_elasticity(ccp_list[:m], coefficients[:m], X[:m], model_labels[:m])

3
Change in : U5 
 [[[-1.74862537e-03 -7.51068954e-05]]

 [[-1.74862537e-03 -7.51068954e-05]]

 [[-1.74862537e-03 -7.51068954e-05]]]
Change in : U6 
 [[[-1.82356086e-03 -7.83255217e-05]]

 [[-1.82356086e-03 -7.83255217e-05]]

 [[-1.82356086e-03 -7.83255217e-05]]]
Change in : e-tron 
 [[[-1.74313296e-03 -7.48709854e-05]]

 [[-1.74313296e-03 -7.48709854e-05]]

 [[-1.74313296e-03 -7.48709854e-05]]]
